# SETTING UP CONNECTION TO SQL SERVER WITH DATASETS

In [ ]:
#importing necessary libraries
import pandas as pd
import sqlite3

#Establishing connection to database
conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",end="")
for table in tables: print(table[0],end=", ")
print("\b\b")

# FIRST PHASE OF TOOL

In [3]:
#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

ALL TABLES: BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA
Enter your zipcode: 07095


## Step 3, filtering formulary_ids by pdp_region_code and contract_id

In [4]:
F_SUBSET = pd.read_sql("select * from PLAN_INFO\
                        where PDP_REGION_CODE is '"+str(user_pdp_region_code)+"'\
                        and CONTRACT_ID like 'S%'",conn)\
           [['CONTRACT_ID','PLAN_ID','SEGMENT_ID','FORMULARY_ID','PREMIUM','DEDUCTIBLE','PDP_REGION_CODE']]
F_SUBSET

,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,PDP_REGION_CODE
0,S0522,4,0,18041,30.4,405.0,4
1,S4802,78,0,18443,36.9,405.0,4
2,S4802,101,0,18439,79.5,0.0,4
3,S5601,8,0,18419,34.3,0.0,4
4,S5601,9,0,18420,84.6,0.0,4
5,S5617,18,0,18082,74.8,405.0,4
6,S5617,249,0,18083,52.9,0.0,4
7,S5660,106,0,18154,34.0,405.0,4
8,S5660,207,0,18155,91.7,350.0,4
9,S5660,220,0,18152,22.6,405.0,4


## Step 4, asking user for drugs and strengths

In [8]:
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):[]\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
user_generics

Enter how many drugs you wish to search: 2
Enter a generic drug name: Lisinopril
Enter a desired strength: 10
Enter a generic drug name: Somatropin
Enter a desired strength: 0


{('LISINOPRIL', '10'): [], ('SOMATROPIN', '0'): []}

## STEP 5

In [ ]:
for drug,strength in user_generics:
    user_generics[(drug,strength)] = \
    pd.read_sql("select PLAN_INFO.CONTRACT_ID, PLAN_INFO.PLAN_ID, FORMULARY_DATA.* \
from (PLAN_INFO join FORMULARY_DATA \
      on instr(FORMULARY_DATA.FORMULARY_ID, PLAN_INFO.FORMULARY_ID) > 0)\
      where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' \
      and PLAN_INFO.CONTRACT_ID like 'S%' \
      and FORMULARY_DATA.RXCUI in (select RXCUI_DATA.RXCUI from RXCUI_DATA\
    where upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and\
    RXCUI_DATA.STRENGTH like '%"+strength+" %')",conn)
#     print ("DRUG: %s\nSTRENGTH: %s\nDATA: \n%s\n%s"%\
#            (drug,strength,"-"*80,user_generics[(drug,strength)]))

## STEP 6

In [107]:
for drug,strength in user_generics:
    user_generics[(drug,strength)] = \
pd.read_sql("select distinct E.FORMULARY_ID, E.RXCUI, BENEFICIARY_COST.CONTRACT_ID, \
BENEFICIARY_COST.PLAN_ID, E.PLAN_NAME, E.PRIOR_AUTHORIZATION_YN, \
BENEFICIARY_COST.TIER, BENEFICIARY_COST.DAYS_SUPPLY, \
BENEFICIARY_COST.COST_AMT_PREF, E.PREMIUM, E.DEDUCTIBLE from \
((select PLAN_INFO.CONTRACT_ID, PLAN_INFO.PLAN_NAME, PLAN_INFO.PLAN_ID, \
PLAN_INFO.PREMIUM, PLAN_INFO.DEDUCTIBLE, FORMULARY_DATA.* \
from (PLAN_INFO join FORMULARY_DATA \
      on instr(FORMULARY_DATA.FORMULARY_ID, PLAN_INFO.FORMULARY_ID) > 0)\
      where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' \
      and PLAN_INFO.CONTRACT_ID like 'S%' \
      and FORMULARY_DATA.RXCUI in (select RXCUI_DATA.RXCUI from RXCUI_DATA\
    where upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and\
    RXCUI_DATA.STRENGTH like '%"+strength+" %')) as E) join BENEFICIARY_COST on \
    (BENEFICIARY_COST.CONTRACT_ID = E.CONTRACT_ID and \
     BENEFICIARY_COST.PLAN_ID = E.PLAN_ID and \
     cast(BENEFICIARY_COST.TIER as integer) = cast(E.TIER_LEVEL_VALUE as integer)) \
where (cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 0 or \
       cast(BENEFICIARY_COST.COVERAGE_LEVEL as integer) = 1) and \
      BENEFICIARY_COST.DAYS_SUPPLY = 1 and\
      cast(BENEFICIARY_COST.COST_TYPE_PREF as integer) = 1",conn)

In [108]:
#EXAMPLE FOR LISINOPRIL
user_generics[('LISINOPRIL','10')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TIER,DAYS_SUPPLY,COST_AMT_PREF,PREMIUM,DEDUCTIBLE
0,00018039,314076,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0
1,00018039,197885,S5820,3,AARP MedicareRx Preferred (PDP),N,1,1,6.0,86.6,0.0
2,00018040,314076,S5921,349,AARP MedicareRx Saver Plus (PDP),N,1,1,1.0,33.2,405.0
3,00018040,197885,S5921,349,AARP MedicareRx Saver Plus (PDP),N,1,1,1.0,33.2,405.0
4,00018041,314076,S0522,4,Symphonix Value Rx (PDP),N,1,1,1.0,30.4,405.0
5,00018041,197885,S0522,4,Symphonix Value Rx (PDP),N,1,1,1.0,30.4,405.0
6,00018043,314076,S5921,386,AARP MedicareRx Walgreens (PDP),N,1,1,0.0,26.8,405.0
7,00018043,197885,S5921,386,AARP MedicareRx Walgreens (PDP),N,1,1,0.0,26.8,405.0
8,00018056,314076,S5810,38,Aetna Medicare Rx Saver (PDP),N,1,1,1.0,34.9,325.0
9,00018056,197885,S5810,38,Aetna Medicare Rx Saver (PDP),N,1,1,1.0,34.9,325.0


## STEP 7, 8A, 8B, 8C

In [109]:
totals = {(drug,strength):None for drug,strength in user_generics}

In [119]:
for key in totals:
    totals[key] = user_generics[key][['FORMULARY_ID','RXCUI','CONTRACT_ID','PLAN_ ID',\
                                     'PLAN_NAME','PRIOR_AUTHORIZATION_YN']]
    totals[key]['TOTAL_ANNUAL_COST'] = user_generics[key]['DEDUCTIBLE'] + \
                                       user_generics[key]['PREMIUM']*12 + \
                                       user_generics[key]['COST_AMT_PREF']*12
    totals[key] = totals[key].sort_values('TOTAL_ANNUAL_COST')[:5] #CHEAPEST 5

/Users/khayyamsaleem/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [120]:
#EXAMPLE WITH LISINOPRIL
totals[('LISINOPRIL','10')]

,FORMULARY_ID,RXCUI,CONTRACT_ID,PLAN_ID,PLAN_NAME,PRIOR_AUTHORIZATION_YN,TOTAL_ANNUAL_COST
37,00018419,197885,S5601,8,SilverScript Choice (PDP),N,447.6
36,00018419,314076,S5601,8,SilverScript Choice (PDP),N,447.6
10,00018057,314076,S5810,278,Aetna Medicare Rx Select (PDP),N,641.4
11,00018057,197885,S5810,278,Aetna Medicare Rx Select (PDP),N,641.4
31,00018255,197885,S5884,150,Humana Walmart Rx Plan (PDP),N,661.8


## TODO: SECOND PHASE OF TOOL

In [ ]:
#CODE FOR STEP 5 AND 6 WITH NDC's INSTEAD OF RxCUI's, FAILED ATTEMPT


#getting NDCs for each drug,strength combo the user offered in step 4
# for drug,strength in user_generics:
#     user_generics[(drug,strength)] = pd.read_sql("select PRODUCTNDC from NDC_DATA\
#                  where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
#                  and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+")",conn)\
#     .applymap(lambda s: "0"*(11-(len(s)-1))+s.replace("-",""))

# for drug,strength in user_generics:
#     print("DRUG: %s, STRENGTH: %s\n%s" % (drug,strength,user_generics[drug,strength]))

# drug = 'LISINOPRIL'
# strength = 10
# pd.read_sql("select * from FORMULARY_DATA\
#  where NDC in (select PRODUCTNDC from NDC_DATA\
#                  where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
#                  and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+"))",conn)
    

# pd.read_sql("select * from FORMULARY_DATA",conn)

# CLEAN_NDC_DATA_TABLE = pd.read_sql("select PRODUCTNDC from NDC_DATA",conn).\
#applymap(lambda s: s.replace("-","")).iloc[1:]
# def clean_ndc(s):
#     if len(s) in [11,10,9]:
        
# CLEAN_NDC_FD_TABLE = pd.read_sql("select NDC from FORMULARY_DATA",conn).applymap()

#RXCUI REWORK
# pd.read_sql("select distinct RXN_DOSE_FORM from RXCUI_DATA",conn)